In [1]:
!pip install googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40712 sha256=fa38a347dc43ad6a8e95654effcdea2342cbdf9418428bef3b888e315e8a5ce8
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


In [2]:
!pip install geopy folium

In [11]:
!pip install folium pandas

In [16]:
import folium
import pandas as pd
from datetime import datetime, timedelta
import random
from IPython.display import display

# Center coordinates for Maharashtra, India
center_coords = [19.7515, 75.7139]

# Create a folium map
maharashtra_map = folium.Map(location=center_coords, zoom_start=7)

# Define toll gates and their coordinates
toll_gates = {
    'Nagpur-Wardha': [(21.1458, 79.0882), (20.7453, 78.6022)],
    'Akola-Amravati': [(20.7096, 77.0082), (20.9374, 77.7796)],
    'Pune-Mumbai': [(18.5204, 73.8567), (19.0760, 72.8777)]
}

# Add toll gates to the map
for route, coords in toll_gates.items():
    folium.Marker(coords[0], tooltip=f'Start of {route}').add_to(maharashtra_map)
    folium.Marker(coords[1], tooltip=f'End of {route}').add_to(maharashtra_map)
    folium.PolyLine(coords, color='blue').add_to(maharashtra_map)

# Display the map
display(maharashtra_map)

# Vehicle types and toll charges
vehicle_types = {
    'car': 50,
    'truck': 100,
    'bike': 20
}

# Generate vehicle IDs
vehicle_ids = [f'VEH{str(i).zfill(4)}' for i in range(1, 101)]

# Generate a simulated dataset
def generate_vehicle_data(num_vehicles, routes, start_time, end_time):
    vehicle_data = []
    for vehicle_id in vehicle_ids[:num_vehicles]:
        vehicle_type = random.choice(list(vehicle_types.keys()))
        route = random.choice(list(routes.keys()))
        start_coords, end_coords = routes[route]
        start_time_vehicle = start_time + timedelta(minutes=random.randint(0, int((end_time - start_time).total_seconds() / 60)))
        speed = random.uniform(40, 100)  # km/h

        # Calculate distance and travel time
        distance = random.uniform(80, 100)  # assuming 100km x 100km area
        travel_time = distance / speed  # hours
        end_time_vehicle = start_time_vehicle + timedelta(hours=travel_time)
        toll_charge = vehicle_types[vehicle_type]

        vehicle_data.append({
            'vehicle_id': vehicle_id,
            'vehicle_type': vehicle_type,
            'route': route,
            'start_coords': start_coords,
            'end_coords': end_coords,
            'start_time': start_time_vehicle,
            'end_time': end_time_vehicle,
            'speed': speed,
            'distance': distance,
            'toll_charge': toll_charge
        })

    return pd.DataFrame(vehicle_data)

# Define simulation parameters
num_vehicles = 100
start_time = datetime.now()
end_time = start_time + timedelta(hours=24)

# Generate vehicle data
vehicle_data = generate_vehicle_data(num_vehicles, toll_gates, start_time, end_time)

# Display the first few rows of the dataset
display(vehicle_data.head())

# Query vehicles on toll roads at a given point in time
def query_vehicles_on_toll(vehicles_df, query_time):
    return vehicles_df[(vehicles_df['start_time'] <= query_time) & (vehicles_df['end_time'] >= query_time)]

# Example query time
query_time = start_time + timedelta(hours=12)

# Get the number of vehicles on toll roads at the query time
vehicles_on_toll = query_vehicles_on_toll(vehicle_data, query_time)

# Display the result
display(vehicles_on_toll)

# Define speed limits for sections (km/h)
speed_limits = {
    'Nagpur-Wardha': 80,
    'Akola-Amravati': 60,
    'Pune-Mumbai': 100
}

# Check speed limits
vehicle_data['within_speed_limit'] = vehicle_data.apply(lambda row: row['speed'] <= speed_limits[row['route']], axis=1)

# Display vehicles not within speed limit
display(vehicle_data[~vehicle_data['within_speed_limit']])


,vehicle_id,vehicle_type,route,start_coords,end_coords,start_time,end_time,speed,distance,toll_charge
0,VEH0001,truck,Akola-Amravati,"(20.7096, 77.0082)","(20.9374, 77.7796)",2024-07-08 11:48:04.004477,2024-07-08 13:02:24.796728,80.275463,99.470045,100
1,VEH0002,truck,Nagpur-Wardha,"(21.1458, 79.0882)","(20.7453, 78.6022)",2024-07-08 09:52:04.004477,2024-07-08 10:48:58.898856,99.005526,93.914837,100
2,VEH0003,car,Akola-Amravati,"(20.7096, 77.0082)","(20.9374, 77.7796)",2024-07-08 08:36:04.004477,2024-07-08 09:32:26.854640,90.353933,84.903838,50
3,VEH0004,car,Pune-Mumbai,"(18.5204, 73.8567)","(19.076, 72.8777)",2024-07-08 17:59:04.004477,2024-07-08 18:54:22.604610,90.236158,83.182701,50
4,VEH0005,car,Pune-Mumbai,"(18.5204, 73.8567)","(19.076, 72.8777)",2024-07-09 01:45:04.004477,2024-07-09 03:45:33.687908,45.387639,91.149517,50


,vehicle_id,vehicle_type,route,start_coords,end_coords,start_time,end_time,speed,distance,toll_charge
16,VEH0017,car,Pune-Mumbai,"(18.5204, 73.8567)","(19.076, 72.8777)",2024-07-08 19:31:04.004477,2024-07-08 21:03:36.584138,59.758507,92.170519,50
28,VEH0029,bike,Pune-Mumbai,"(18.5204, 73.8567)","(19.076, 72.8777)",2024-07-08 19:23:04.004477,2024-07-08 20:32:37.191682,74.007791,85.791213,20
65,VEH0066,bike,Pune-Mumbai,"(18.5204, 73.8567)","(19.076, 72.8777)",2024-07-08 19:01:04.004477,2024-07-08 21:28:22.683916,40.072248,98.384931,20


,vehicle_id,vehicle_type,route,start_coords,end_coords,start_time,end_time,speed,distance,toll_charge,within_speed_limit
0,VEH0001,truck,Akola-Amravati,"(20.7096, 77.0082)","(20.9374, 77.7796)",2024-07-08 11:48:04.004477,2024-07-08 13:02:24.796728,80.275463,99.470045,100,False
1,VEH0002,truck,Nagpur-Wardha,"(21.1458, 79.0882)","(20.7453, 78.6022)",2024-07-08 09:52:04.004477,2024-07-08 10:48:58.898856,99.005526,93.914837,100,False
2,VEH0003,car,Akola-Amravati,"(20.7096, 77.0082)","(20.9374, 77.7796)",2024-07-08 08:36:04.004477,2024-07-08 09:32:26.854640,90.353933,84.903838,50,False
6,VEH0007,car,Akola-Amravati,"(20.7096, 77.0082)","(20.9374, 77.7796)",2024-07-09 05:14:04.004477,2024-07-09 06:28:25.709924,71.488698,88.600421,50,False
7,VEH0008,bike,Akola-Amravati,"(20.7096, 77.0082)","(20.9374, 77.7796)",2024-07-09 04:53:04.004477,2024-07-09 06:02:04.641408,79.330752,91.244401,20,False
10,VEH0011,bike,Nagpur-Wardha,"(21.1458, 79.0882)","(20.7453, 78.6022)",2024-07-08 14:52:04.004477,2024-07-08 15:42:36.412941,95.929885,80.805165,20,False
12,VEH0013,bike,Akola-Amravati,"(20.7096, 77.0082)","(20.9374, 77.7796)",2024-07-09 01:27:04.004477,2024-07-09 02:39:49.007108,69.875612,84.724230,20,False
17,VEH0018,bike,Akola-Amravati,"(20.7096, 77.0082)","(20.9374, 77.7796)",2024-07-08 09:47:04.004477,2024-07-08 10:49:56.235753,93.277859,97.740460,20,False
18,VEH0019,truck,Nagpur-Wardha,"(21.1458, 79.0882)","(20.7453, 78.6022)",2024-07-09 01:50:04.004477,2024-07-09 02:46:54.489856,98.345986,93.168763,100,False
24,VEH0025,bike,Akola-Amravati,"(20.7096, 77.0082)","(20.9374, 77.7796)",2024-07-08 09:47:04.004477,2024-07-08 10:37:22.650403,97.952818,82.134687,20,False
